<a href="https://colab.research.google.com/github/jeanmaxcacacho/199.X-Mformer/blob/main/Mformer_replicate.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### TODOs in this notebook:
*   get sample data from our actual dataset
*   instantiate all Mformer foundations
*   obtain moral foundation measurements for all foundations

In [13]:
from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification
import torch

FOUNDATIONS = [
    "authority",
    "care",
    "fairness",
    "loyalty",
    "sanctity"
]

MODEL_FOUNDATIONS = []
for F in FOUNDATIONS:
  MODEL_FOUNDATIONS.append(f"joshnguyen/mformer-{F}")

"""
Pretrained models locked and loaded in MODELS as:
(tokenizer, model)

[0] - [4] respectively
1. authority
2. care
3. fairness
4. loyalty
5. sanctity
"""

MODELS = []
for MODEL_FOUNDATION in MODEL_FOUNDATIONS:
  MODELS.append((
      AutoTokenizer.from_pretrained(MODEL_FOUNDATION),
      AutoModelForSequenceClassification.from_pretrained(
          MODEL_FOUNDATION,
          device_map="auto"
      )
  ))

In [5]:
"""
Load 5 texts from the dataset: https://www.kaggle.com/datasets/asaniczka/reddit-on-israel-palestine-daily-updated
...then perform inference

Sample texts were obtained arbitrarily from the preview table, these were not properly sampled!
"""

instances = [
  "who confiscates a small child's bike? nazis",
  "Why aren't they deliberately hitting schools and hospitals? Amateurs!",
  "What are you expecting? That the UN will declare a war on a country to save the sorry hide of hamas?",
  "Superman says NO to genocide!",
  "Only criminals hide their faces. Hope they het the pineapple treatment on hell"
]

In [14]:
# AUTHORITY
authority_tokenizer = MODELS[0][0]
authority_classifier = MODELS[0][1]

authority_inputs = authority_tokenizer(
    instances,
    padding=True,
    truncation=True,
    return_tensors='pt'
).to(authority_classifier.device)

authority_classifier.eval()
with torch.no_grad():
  authority_outputs = authority_classifier(**authority_inputs)

authority_probs = torch.softmax(authority_outputs.logits, dim=1)
authority_probs = authority_probs[:, 1]
authority_probs = authority_probs.detach().cpu().numpy()

print(f"Probability of foundation 'AUTHORITY':", "\n")
for instance, prob in zip(instances, authority_probs):
    print(instance, "->", prob, "\n")

Probability of foundation 'AUTHORITY': 

who confiscates a small child's bike? nazis -> 0.3475795 

Why aren't they deliberately hitting schools and hospitals? Amateurs! -> 0.2956997 

What are you expecting? That the UN will declare a war on a country to save the sorry hide of hamas? -> 0.27424964 

Superman says NO to genocide! -> 0.22523473 

Only criminals hide their faces. Hope they het the pineapple treatment on hell -> 0.22267361 



In [16]:
# CARE
care_tokenizer = MODELS[1][0]
care_classifier = MODELS[1][1]

care_inputs = care_tokenizer(
    instances,
    padding=True,
    truncation=True,
    return_tensors='pt'
).to(care_classifier.device)

care_classifier.eval()
with torch.no_grad():
  care_outputs = care_classifier(**care_inputs)

care_probs = torch.softmax(care_outputs.logits, dim=1)
care_probs = care_probs[:, 1]
care_probs = care_probs.detach().cpu().numpy()

print(f"Probability of foundation 'CARE':", "\n")
for instance, prob in zip(instances, care_probs):
    print(instance, "->", prob, "\n")

Probability of foundation 'CARE': 

who confiscates a small child's bike? nazis -> 0.3594455 

Why aren't they deliberately hitting schools and hospitals? Amateurs! -> 0.19800054 

What are you expecting? That the UN will declare a war on a country to save the sorry hide of hamas? -> 0.561702 

Superman says NO to genocide! -> 0.8456664 

Only criminals hide their faces. Hope they het the pineapple treatment on hell -> 0.31601465 



In [17]:
# FAIRNESS
fairness_tokenizer = MODELS[2][0]
fairness_classifier = MODELS[2][1]

fairness_inputs = fairness_tokenizer(
    instances,
    padding=True,
    truncation=True,
    return_tensors='pt'
).to(fairness_classifier.device)

fairness_classifier.eval()
with torch.no_grad():
  fairness_outputs = fairness_classifier(**fairness_inputs)

fairness_probs = torch.softmax(fairness_outputs.logits, dim=1)
fairness_probs = fairness_probs[:, 1]
fairness_probs = fairness_probs.detach().cpu().numpy()

print(f"Probability of foundation 'FAIRNESS':", "\n")
for instance, prob in zip(instances, fairness_probs):
    print(instance, "->", prob, "\n")

Probability of foundation 'FAIRNESS': 

who confiscates a small child's bike? nazis -> 0.67864925 

Why aren't they deliberately hitting schools and hospitals? Amateurs! -> 0.24773857 

What are you expecting? That the UN will declare a war on a country to save the sorry hide of hamas? -> 0.19757043 

Superman says NO to genocide! -> 0.53763705 

Only criminals hide their faces. Hope they het the pineapple treatment on hell -> 0.50723505 



In [19]:
# LOYALTY
loyalty_tokenizer = MODELS[3][0]
loyalty_classifier = MODELS[3][1]

loyalty_inputs = loyalty_tokenizer(
    instances,
    padding=True,
    truncation=True,
    return_tensors='pt'
).to(loyalty_classifier.device)

loyalty_classifier.eval()
with torch.no_grad():
  loyalty_outputs = loyalty_classifier(**loyalty_inputs)

loyalty_probs = torch.softmax(loyalty_outputs.logits, dim=1)
loyalty_probs = loyalty_probs[:, 1]
loyalty_probs = loyalty_probs.detach().cpu().numpy()

print(f"Probability of foundation 'LOYALTY':", "\n")
for instance, prob in zip(instances, loyalty_probs):
    print(instance, "->", prob, "\n")

Probability of foundation 'LOYALTY': 

who confiscates a small child's bike? nazis -> 0.019190745 

Why aren't they deliberately hitting schools and hospitals? Amateurs! -> 0.0067726853 

What are you expecting? That the UN will declare a war on a country to save the sorry hide of hamas? -> 0.0074807405 

Superman says NO to genocide! -> 0.15861931 

Only criminals hide their faces. Hope they het the pineapple treatment on hell -> 0.024015982 



In [20]:
# SANCTITY
sanctity_tokenizer = MODELS[4][0]
sanctity_classifier = MODELS[4][1]

sanctity_inputs = sanctity_tokenizer(
    instances,
    padding=True,
    truncation=True,
    return_tensors='pt'
).to(sanctity_classifier.device)

sanctity_classifier.eval()
with torch.no_grad():
  sanctity_outputs = sanctity_classifier(**sanctity_inputs)

sanctity_probs = torch.softmax(sanctity_outputs.logits, dim=1)
sanctity_probs = sanctity_probs[:, 1]
sanctity_probs = sanctity_probs.detach().cpu().numpy()

print(f"Probability of foundation 'SANCTITY':", "\n")
for instance, prob in zip(instances, sanctity_probs):
    print(instance, "->", prob, "\n")

Probability of foundation 'SANCTITY': 

who confiscates a small child's bike? nazis -> 0.07356864 

Why aren't they deliberately hitting schools and hospitals? Amateurs! -> 0.19583298 

What are you expecting? That the UN will declare a war on a country to save the sorry hide of hamas? -> 0.18396676 

Superman says NO to genocide! -> 0.30274275 

Only criminals hide their faces. Hope they het the pineapple treatment on hell -> 0.31580466 

